<a href="https://colab.research.google.com/github/aim56009/AI-MA_project/blob/main/Chew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install partitura
import partitura
import numpy as np
import os
import pickle
import torch
from sklearn.metrics import accuracy_score
from collections import defaultdict

In [1]:
!git clone https://github.com/aim56009/AI-MA_project.git

Cloning into 'AI-MA_project'...
remote: Enumerating objects: 2950, done.
remote: Counting objects: 100% (2950/2950), done.
remote: Compressing objects: 100% (829/829), done.
remote: Total 2950 (delta 2200), reused 2803 (delta 2119), pack-reused 0
Receiving objects: 100% (2950/2950), 4.87 MiB | 6.11 MiB/s, done.
Resolving deltas: 100% (2200/2200), done.
Checking out files: 100% (3184/3184), done.


# Chew


In [ ]:
def evaluation_chew():

    history = defaultdict(list)
    path = "AI-MA_project/chorales_converted"

    ### load the chorales in xml format ###
    for filename in os.listdir(path):
        if not filename.endswith('.xml'): continue
        fullname = os.path.join(path, filename)
        

        part = partitura.load_musicxml(fullname)
            

        ### apply chews method ### 
        chew_sep = partitura.musicanalysis.estimate_voices(part, monophonic_voices=True)


        ### seperate the results -> e.g. pos_zero = all positions where chew prediction says voice 0 ####
        pos_zero = np.where(chew_sep==1)
        pos_one = np.where(chew_sep==2)
        pos_two = np.where(chew_sep==3)
        pos_three = np.where(chew_sep==4)


        ### create notearray object that contain only the corresponding voice ###
        note_array_zero = partitura.utils.ensure_notearray(part)[pos_zero]
        note_array_one = partitura.utils.ensure_notearray(part)[pos_one]
        note_array_two = partitura.utils.ensure_notearray(part)[pos_two]
        note_array_three = partitura.utils.ensure_notearray(part)[pos_three]


        ### create pr representation of all voices ###
        pr_zero = partitura.utils.compute_pianoroll(note_array_zero,time_unit = "beat",time_div = 12)
        pr_zero = pr_zero.toarray()

        pr_one = partitura.utils.compute_pianoroll(note_array_one,time_unit = "beat",time_div = 12)
        pr_one = pr_one.toarray()

        pr_two = partitura.utils.compute_pianoroll(note_array_two,time_unit = "beat",time_div = 12)
        pr_two = pr_two.toarray()

        pr_three = partitura.utils.compute_pianoroll(note_array_three,time_unit = "beat",time_div = 12)
        pr_three = pr_three.toarray()


        ### get original data (as labels) ###
        part_zero = part[0]
        part_one = part[1]
        part_two = part[2]
        part_three = part[3]


        ### compute pianoroll representation of original data for computing acc ###
        pr_zero_ori = partitura.utils.compute_pianoroll(part_zero, time_unit = "beat",time_div = 12)
        v0 = pr_zero_ori.toarray()

        pr_one_ori = partitura.utils.compute_pianoroll(part_one, time_unit = "beat",time_div = 12)
        v1 = pr_one_ori.toarray()

        pr_two_ori = partitura.utils.compute_pianoroll(part_two, time_unit = "beat",time_div = 12)
        v2 = pr_two_ori.toarray()

        pr_three_ori = partitura.utils.compute_pianoroll(part_three, time_unit = "beat",time_div = 12)
        v3 = pr_three_ori.toarray()

        if pr_zero.shape == pr_one.shape == pr_two.shape == pr_three.shape  :
            ### compute accuracy ###
            acc_v0 = accuracy_score(torch.tensor(pr_zero), v0)
            acc_v1 = accuracy_score(torch.tensor(pr_one), v1)
            acc_v2 = accuracy_score(torch.tensor(pr_two),v2)
            acc_v3 = accuracy_score(torch.tensor(pr_three), v3)

            history["accuracy_v0"].append(acc_v0)
            history["accuracy_v1"].append(acc_v1)
            history["accuracy_v2"].append(acc_v2)
            history["accuracy_v3"].append(acc_v3)
            
        else: print("pr shape does not match for:", filename)
        
    return history

In [ ]:
hist_acc =  evaluation_chew()

pr shape does not match for: chor245.xml
pr shape does not match for: chor264.xml
pr shape does not match for: chor286.xml
pr shape does not match for: chor056.xml
pr shape does not match for: chor283.xml
pr shape does not match for: chor074.xml
pr shape does not match for: chor043.xml
pr shape does not match for: chor252.xml
pr shape does not match for: chor133.xml
pr shape does not match for: chor095.xml


In [ ]:
total_len = len(hist_acc["accuracy_v0"])
total_len

360

In [ ]:
accuracy_v0 = np.sum(hist_acc["accuracy_v0"])/total_len
accuracy_v1 = np.sum(hist_acc["accuracy_v1"])/total_len
accuracy_v2 = np.sum(hist_acc["accuracy_v2"])/total_len
accuracy_v3 = np.sum(hist_acc["accuracy_v3"])/total_len

print("accuracy of v0-v3:",accuracy_v0,accuracy_v1,accuracy_v2,accuracy_v3)

accuracy of v0-v3: 0.9882161458333333 0.9801215277777777 0.9844618055555555 0.9959201388888889


# HMM approach

In [ ]:
## This block was to save the xml files to midi and download it to put it in the right folder on github ###
"""
#path = "AI-MA_project/chorales_converted"
path = "chorales_converted"


for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)
    part = partitura.load_musicxml(fullname)
    try:
      partitura.save_score_midi(part,out= "midi_test/"+filename[:8]+"mid",part_voice_assign_mode=4)
    except:
      print(filename[:8]+"mid")


#path = "midi_test/"
#for filename in os.listdir(path):
#    files.download(path + filename) 

#!zip -r /content/file.zip midi_test
#files.download("/content/midi_zip_file.zip")
"""

HMM loop

In [49]:
def evaluation_hmm():
    history_ = defaultdict(list)

    path = "AI-MA_project/hmm_sep"

    for filename in os.listdir(path):
        if not filename.endswith('.mid'): continue
        fullname = os.path.join(path, filename)

        part_hmm = partitura.load_score_midi(fullname,part_voice_assign_mode=2)
        voice_info = partitura.utils.note_array_from_part(part_hmm)["voice"]

        ### seperate the results -> e.g. pos_zero = all positions where chew prediction says voice 0 ####
        pos_0 = np.where(voice_info==1)
        pos_1 = np.where(voice_info==2)
        pos_2 = np.where(voice_info==3)
        pos_3 = np.where(voice_info==4)

        ### create notearray object that contain only the corresponding voice ###
        note_array_0= partitura.utils.ensure_notearray(part_hmm)[pos_0]
        note_array_1 = partitura.utils.ensure_notearray(part_hmm)[pos_1]
        note_array_2 = partitura.utils.ensure_notearray(part_hmm)[pos_2]
        note_array_3 = partitura.utils.ensure_notearray(part_hmm)[pos_3]

        if len(note_array_3) !=0 and len(note_array_2) !=0 and len(note_array_1) !=0 and len(note_array_0) !=0:
            ### create pr representation of all voices ###
            pr_0 = partitura.utils.compute_pianoroll(note_array_0,time_unit = "beat",time_div = 12)
            pr_0 = pr_0.toarray()

            pr_1 = partitura.utils.compute_pianoroll(note_array_1,time_unit = "beat",time_div = 12)
            pr_1 = pr_1.toarray()

            pr_2 = partitura.utils.compute_pianoroll(note_array_2,time_unit = "beat",time_div = 12)
            pr_2 = pr_2.toarray()

            pr_3 = partitura.utils.compute_pianoroll(note_array_3,time_unit = "beat",time_div = 12)
            pr_3 = pr_3.toarray()


            fullname_ = os.path.join("AI-MA_project/chorales_converted", filename[:8] + "xml")
            part_ = partitura.load_musicxml(fullname_)

            ### get original data (as labels) ###
            part_zero = part_[0]
            part_one = part_[1]
            part_two = part_[2]
            part_three = part_[3]


            ### compute pianoroll representation of original data for computing acc ###
            pr_zero_ori = partitura.utils.compute_pianoroll(part_zero, time_unit = "beat",time_div = 12)
            v0 = pr_zero_ori.toarray()

            pr_one_ori = partitura.utils.compute_pianoroll(part_one, time_unit = "beat",time_div = 12)
            v1 = pr_one_ori.toarray()

            pr_two_ori = partitura.utils.compute_pianoroll(part_two, time_unit = "beat",time_div = 12)
            v2 = pr_two_ori.toarray()

            pr_three_ori = partitura.utils.compute_pianoroll(part_three, time_unit = "beat",time_div = 12)
            v3 = pr_three_ori.toarray()

            if pr_0.shape == pr_1.shape == pr_2.shape == pr_3.shape  :
                ### compute accuracy ###
                acc_v0 = accuracy_score(torch.tensor(pr_0), v0)
                acc_v1 = accuracy_score(torch.tensor(pr_1), v1)
                acc_v2 = accuracy_score(torch.tensor(pr_2),v2)
                acc_v3 = accuracy_score(torch.tensor(pr_3), v3)

                history_["accuracy_v0"].append(acc_v0)
                history_["accuracy_v1"].append(acc_v1)
                history_["accuracy_v2"].append(acc_v2)
                history_["accuracy_v3"].append(acc_v3)
            else:
              print("size does not match",filename)
        
    return history_

In [50]:
hist_acc_hmm =  evaluation_hmm()

size does not match chor283.mid
size does not match chor043.mid


In [51]:
total_len_hmm = len(hist_acc_hmm["accuracy_v0"])

accuracy_v0_hmm = np.sum(hist_acc_hmm["accuracy_v0"])/total_len_hmm
accuracy_v1_hmm = np.sum(hist_acc_hmm["accuracy_v1"])/total_len_hmm
accuracy_v2_hmm = np.sum(hist_acc_hmm["accuracy_v2"])/total_len_hmm
accuracy_v3_hmm = np.sum(hist_acc_hmm["accuracy_v3"])/total_len_hmm

print("accuracy of v0-v3 -HMM:",accuracy_v0_hmm,accuracy_v1_hmm,accuracy_v2_hmm,accuracy_v3_hmm)

accuracy of v0-v3 -HMM: 0.99189453125 0.982470703125 0.987744140625 0.9976806640625


# Try single files

In [30]:
part_hmm = partitura.load_score_midi("AI-MA_project/hmm_sep_t/chor002.mid",part_voice_assign_mode=2)
voice_info = partitura.utils.note_array_from_part(part_hmm)["voice"]
voice_info

array([1, 2, 3, 4, 1, 1, 2, 3, 4, 1, 2, 3, 4, 2, 1, 2, 3, 4, 2, 1, 2, 3,
       4, 1, 2, 3, 4, 1, 3, 4, 2, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1,
       2, 3, 4, 3, 1, 2, 3, 4, 1, 4, 1, 4, 1, 2, 4, 1, 3, 4, 2, 1, 2, 3,
       4, 1, 2, 3, 4, 3, 1, 2, 3, 4, 1, 2, 1, 2, 3, 4, 2, 1, 2, 3, 4, 1,
       2, 3, 4, 1, 3, 1, 2, 3, 4, 3, 4, 1, 2, 3, 4, 1, 1, 2, 3, 4, 1, 2,
       3, 4, 1, 2, 3, 4, 2, 3, 1, 3, 4, 2, 1, 2, 3, 4, 1, 3, 1, 2, 3, 4,
       1, 2, 3, 4, 1, 2, 3, 4, 3, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 1, 2, 3,
       4, 2, 1, 2, 3, 4, 3, 1, 2, 3, 4, 1, 2, 1, 2, 4, 1, 2, 4, 1, 2, 3,
       4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 1, 2, 3, 4, 2, 1, 2, 3, 4, 1,
       1, 2, 3, 2, 3, 4, 2, 3, 4, 1, 4, 1, 4, 2, 3, 1, 2, 3, 4, 1, 2, 1,
       2, 3, 4, 2, 2, 1, 2, 3, 4], dtype=int32)

In [31]:
### seperate the results -> e.g. pos_zero = all positions where chew prediction says voice 0 ####
pos_0 = np.where(voice_info==1)
pos_1 = np.where(voice_info==2)
pos_2 = np.where(voice_info==3)
pos_3 = np.where(voice_info==4)

In [32]:
### create notearray object that contain only the corresponding voice ###
note_array_0= partitura.utils.ensure_notearray(part_hmm)[pos_0]
note_array_1 = partitura.utils.ensure_notearray(part_hmm)[pos_1]
note_array_2 = partitura.utils.ensure_notearray(part_hmm)[pos_2]
note_array_3 = partitura.utils.ensure_notearray(part_hmm)[pos_3]

In [33]:
len(note_array_3)

52

In [34]:
### create pr representation of all voices ###
pr_0 = partitura.utils.compute_pianoroll(note_array_0,time_unit = "beat",time_div = 12)
pr_0 = pr_0.toarray()

pr_1 = partitura.utils.compute_pianoroll(note_array_1,time_unit = "beat",time_div = 12)
pr_1 = pr_1.toarray()

pr_2 = partitura.utils.compute_pianoroll(note_array_2,time_unit = "beat",time_div = 12)
pr_2 = pr_2.toarray()

pr_3 = partitura.utils.compute_pianoroll(note_array_3,time_unit = "beat",time_div = 12)
pr_3 = pr_3.toarray()

In [35]:
pr_0.shape,pr_1.shape , pr_2.shape , pr_3.shape

((128, 624), (128, 624), (128, 624), (128, 624))

### original pr of chor001

In [82]:
path = "AI-MA_project/chorales_converted"

fullname = os.path.join(path, "chor001.xml")
part = partitura.load_musicxml(fullname)

### get original data (as labels) ###
part_zero = part[0]
part_one = part[1]
part_two = part[2]
part_three = part[3]


### compute pianoroll representation of original data for computing acc ###
pr_zero_ori = partitura.utils.compute_pianoroll(part_zero, time_unit = "beat",time_div = 12)
v0 = pr_zero_ori.toarray()

pr_one_ori = partitura.utils.compute_pianoroll(part_one, time_unit = "beat",time_div = 12)
v1 = pr_one_ori.toarray()

pr_two_ori = partitura.utils.compute_pianoroll(part_two, time_unit = "beat",time_div = 12)
v2 = pr_two_ori.toarray()

pr_three_ori = partitura.utils.compute_pianoroll(part_three, time_unit = "beat",time_div = 12)
v3 = pr_three_ori.toarray()

In [84]:
v0.shape,v1.shape,v2.shape,v3.shape

((128, 756), (128, 756), (128, 756), (128, 756))

In [85]:
if pr_0.shape == pr_1.shape == pr_2.shape == pr_3.shape  :
    ### compute accuracy ###
    acc_v0 = accuracy_score(torch.tensor(pr_0), v0)
    acc_v1 = accuracy_score(torch.tensor(pr_1), v1)
    acc_v2 = accuracy_score(torch.tensor(pr_2),v2)
    acc_v3 = accuracy_score(torch.tensor(pr_3), v3)

    history["accuracy_v0"].append(acc_v0)
    history["accuracy_v1"].append(acc_v1)
    history["accuracy_v2"].append(acc_v2)
    history["accuracy_v3"].append(acc_v3)
else:
  print("size does not match")

size does not match


In [34]:
pr_0.shape,pr_1.shape , pr_2.shape , pr_3.shape

((128, 756), (128, 720), (128, 708), (128, 90))